# Hacemos la llamada a la API para obtener datos del clima desde el año 2014 hasta el actual (2024)

- En primer lugar creamos una función (fetch_data) para consumir la api, esta función necesita de tres parámetros: **start_str(fecha de inicio), end_str(fecha final) y token**, la clave api key con la que accedemos. Esta petición nos devuelve el siguiente formato:

        b'{\n  "descripcion" : "exito",\n  "estado" : 200,\n  "datos" : "https://opendata.aemet.es/opendata/sh/161db1f3",\n  "metadatos" : "https://opendata.aemet.es/opendata/sh/b3aa9d28"\n}'


- Devolvemos el valor de la clave **"datos"**, ya que en esa url se encuentran los datos que queremos obtener en formato Json

- Luego en la función fetch_and_convert_data accedemos a la esta url, obtenemos los datos en formato json y los normalizamos para guardarlos en un dataframe de pandas

- Por último tenemos la función fetch_and_combine_data, a la que le indicamos en sus parámetros la fecha de comienzo y de final total para la extracción de los datos. Para ello hemos accedido por años, ya que la API bloquea las llamadas cada cierto tiempo/volumen de llamadas. Dentro de la función se divide el año en lotes de 15 días iguales, y se llama a las dos funciones anteriores para obtener los csv correspondientes a cada fracción del tiempo que le pasemos. Itera hasta terminar el año, y mientras va iterando va guardando y uniendo los csv en una variable local llamada **all_data_frames**, a la que se llama en último lugar porque es la que contiene el csv anual total. Una vez termina de iterar sobre las fechas correspondientes **se guarda el archivo en formato .csv con el año correspondiente.**

In [ ]:
import http.client
import requests
import datetime
import time
import pandas as pd

# Función para obtener las URLs desde la API de la AEMET
def fetch_data(start_date, end_date):
    conn = http.client.HTTPSConnection("opendata.aemet.es")
    
    # Formatear las fechas a formato adecuado para la URL
    start_str = start_date.strftime("%Y-%m-%dT%H%%3A%M%%3A%SUTC")
    end_str = end_date.strftime("%Y-%m-%dT%H%%3A%M%%3A%SUTC")
    
    headers = {
        'cache-control': "no-cache"
    }
    
    token = ""  # Aquí hay que indicarle el token de acceso que se obtiene en el siguiente enlace https://opendata.aemet.es/centrodedescargas/altaUsuario?
    
    # Realizar la solicitud para obtener las URLs de los datos
    conn.request("GET", f"/opendata/api/valores/climatologicos/diarios/datos/fechaini/{start_str}/fechafin/{end_str}/todasestaciones/?api_key={token}", headers=headers)
    
    res = conn.getresponse()
    data = res.read()
    
    # Decodificar la respuesta
    json_response = eval(data.decode("utf-8"))
    
    # Extraer la URL de los datos
    data_url = json_response["datos"]
    return data_url

# Función para descargar los datos desde la URL proporcionada
def fetch_and_convert_data(data_url):
    response = requests.get(data_url)
    
    # Si la solicitud fue exitosa (status 200)
    if response.status_code == 200:
        json_data = response.json()  # Convertir la respuesta en JSON
        df = pd.json_normalize(json_data)  # Convertir los datos en un DataFrame
        return df
    else:
        print(f"Error al obtener los datos de {data_url}")
        return None

# Función para dividir el rango de fechas en bloques de 15 días y obtener las URLs automáticamente
def fetch_and_combine_data(start_date, end_date):
    all_data_frames = []  # Lista para almacenar los DataFrames

    current_start = start_date
    while current_start < end_date:
        current_end = current_start + datetime.timedelta(days=14)  # Bloque de 15 días
        if current_end > end_date:
            current_end = end_date
        
        print(f"Obteniendo datos desde {current_start} hasta {current_end}")
        
        # Obtener la URL de los datos para el bloque actual
        data_url = fetch_data(current_start, current_end)
        
        # Descargar y convertir los datos de la URL
        df = fetch_and_convert_data(data_url)
        if df is not None:
            all_data_frames.append(df)  # Añadir el DataFrame a la lista

        # Esperar 6 segundos entre solicitudes para evitar el bloqueo
        time.sleep(3)
        
        # Actualizar la fecha de inicio para el siguiente bloque
        current_start = current_end + datetime.timedelta(days=1)

    # Concatenar todos los DataFrames en uno solo
    if all_data_frames:
        full_df = pd.concat(all_data_frames, ignore_index=True)
        
        # Guardar el DataFrame como archivo CSV
        full_df.to_csv(f"DatosClimatologicos_{start_date.year}.csv", index=False)
        print(f"Archivo 'DatosClimatologicos_{start_date.year}.csv' guardado con éxito")
    else:
        print("No se han podido obtener los datos")

# Fechas de inicio y fin del rango
start_date = datetime.datetime(2023, 1, 1)
end_date = datetime.datetime(2023, 12, 31)

# Obtener los datos en intervalos de 15 días y guardarlos en un CSV
fetch_and_combine_data(start_date, end_date)